<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Colmar 

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Colmar_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Colmar - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=ind

In [7]:
class ColmarSpider(scrapy.Spider) :
    # Name of spider
    name = 'Colmar'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuOdw5QGwAIB0gIkMTkyZGI1ODItMzY2My00YjFmLTgxYTgtNTQxM2JhMmQ3MzE52AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=colmar&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=colmar&search_pageview_id=383d5d311d4c0321&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(ColmarSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
colmar = pd.read_json('results_hotels/Colmar_multipage.json')

In [11]:
len(colmar)

125

In [12]:
colmar.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/logement-peti...,"Logement Petite Venise ""appartement & parking""","9,1",Hébergement géré par un particulier
1,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/le-victor-hug...,Le Victor Hugo,"8,6",Hébergement géré par un particulier
2,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/bredala.fr.ht...,Appartement Bredala,"8,6","L'hébergement Bredala se situe à Colmar, à 100..."
3,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/au-brochet.fr...,AU brochet,"8,4",Hébergement géré par un particulier
4,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/cote-jardin-c...,Gite Côté Jardin Colmar,"8,8",Hébergement géré par un particulier
5,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/quatorze.fr.h...,Hôtel Quatorze,"8,2",L'Hôtel Quatorze est un établissement design s...
6,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/gite-des-bain...,Gîte des Bains PARKING HYPERCENTRE 6-8personnes,"7,9",Hébergement géré par un particulier
7,Colmar,https://www.booking.com/hotel/fr/le-riesling-c...,Le Riesling Colmar All inclusive Free parking,"9,5",Hébergement géré par un particulier
8,Colmar,https://www.booking.com/hotel/fr/chez-thomas-c...,Chez Thomas,"7,3","Offrant une vue sur la ville, l'hébergement Ch..."
9,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/sous-les-till...,Sous les Tilleuls PARKING HYPERCENTRE 4-6 Pers...,"8,4",Hébergement géré par un particulier


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in colmar['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
colmar['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in colmar['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
colmar['latitude'] = lat
colmar['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in colmar['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
colmar['hotel_description_long'] = result_long_description

In [20]:
colmar.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/logement-peti...,"Logement Petite Venise ""appartement & parking""","9,1",Hébergement géré par un particulier,"48.07366420,7.35776200",48.07366420,7.35776200,\nVous pouvez bénéficier d'une réduction Geniu...
1,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/le-victor-hug...,Le Victor Hugo,"8,6",Hébergement géré par un particulier,"48.07611700,7.36353500",48.07611700,7.36353500,\nVous pouvez bénéficier d'une réduction Geniu...
2,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/bredala.fr.ht...,Appartement Bredala,"8,6","L'hébergement Bredala se situe à Colmar, à 100...","48.07805253,7.35612649",48.07805253,7.35612649,\nVous pouvez bénéficier d'une réduction Geniu...
3,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/au-brochet.fr...,AU brochet,"8,4",Hébergement géré par un particulier,"48.07406922,7.35810319",48.07406922,7.35810319,\nVous pouvez bénéficier d'une réduction Geniu...
4,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/cote-jardin-c...,Gite Côté Jardin Colmar,"8,8",Hébergement géré par un particulier,"48.07990300,7.36100800",48.07990300,7.36100800,\nVous pouvez bénéficier d'une réduction Geniu...
5,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/quatorze.fr.h...,Hôtel Quatorze,"8,2",L'Hôtel Quatorze est un établissement design s...,"48.07623850,7.35644773",48.07623850,7.35644773,\nVous pouvez bénéficier d'une réduction Geniu...
6,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/gite-des-bain...,Gîte des Bains PARKING HYPERCENTRE 6-8personnes,"7,9",Hébergement géré par un particulier,"48.07984157,7.35429400",48.07984157,7.35429400,\nVous pouvez bénéficier d'une réduction Geniu...
7,Colmar,https://www.booking.com/hotel/fr/le-riesling-c...,Le Riesling Colmar All inclusive Free parking,"9,5",Hébergement géré par un particulier,"48.07984425,7.36613929",48.07984425,7.36613929,\nVous pouvez bénéficier d'une réduction Geniu...
8,Colmar,https://www.booking.com/hotel/fr/chez-thomas-c...,Chez Thomas,"7,3","Offrant une vue sur la ville, l'hébergement Ch...","48.08231130,7.34654790",48.08231130,7.34654790,\nVous pouvez bénéficier d'une réduction Geniu...
9,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/sous-les-till...,Sous les Tilleuls PARKING HYPERCENTRE 4-6 Pers...,"8,4",Hébergement géré par un particulier,"48.07831426,7.35740174",48.07831426,7.35740174,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
colmar['hotel_description_long'] = colmar['hotel_description_long'].replace('\n','', regex=True)

In [22]:
colmar.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/logement-peti...,"Logement Petite Venise ""appartement & parking""","9,1",Hébergement géré par un particulier,"48.07366420,7.35776200",48.07366420,7.35776200,Vous pouvez bénéficier d'une réduction Genius ...
1,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/le-victor-hug...,Le Victor Hugo,"8,6",Hébergement géré par un particulier,"48.07611700,7.36353500",48.07611700,7.36353500,Vous pouvez bénéficier d'une réduction Genius ...
2,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/bredala.fr.ht...,Appartement Bredala,"8,6","L'hébergement Bredala se situe à Colmar, à 100...","48.07805253,7.35612649",48.07805253,7.35612649,Vous pouvez bénéficier d'une réduction Genius ...
3,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/au-brochet.fr...,AU brochet,"8,4",Hébergement géré par un particulier,"48.07406922,7.35810319",48.07406922,7.35810319,Vous pouvez bénéficier d'une réduction Genius ...
4,"Centre-ville de Colmar, Colmar",https://www.booking.com/hotel/fr/cote-jardin-c...,Gite Côté Jardin Colmar,"8,8",Hébergement géré par un particulier,"48.07990300,7.36100800",48.07990300,7.36100800,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
colmar['hotel_description_long'][55]

"Vous pouvez bénéficier d'une réduction Genius dans l'établissement Aux Bijoux de Colmar - Le Saphir - 1 PARKING GRATUIT\xa0! Connectez-vous pour économiser.Situé à Colmar, à seulement 2,1 km de la Maison des Têtes, l'établissement Aux Bijoux de Colmar - Le Saphir - 1 PARKING GRATUIT propose un hébergement avec vue sur la ville, connexion Wi-Fi et parking privé gratuits. Il se trouve à 600 mètres de la petite Venise de Colmar et à 1,8 km du théâtre de Colmar.Doté d'un lecteur Blu-ray, l'appartement dispose d'une cuisine avec un lave-vaisselle, un micro-ondes et un réfrigérateur, d'un salon, d'un coin repas, de 3 chambres et de 2 salles de bains avec douche. Les serviettes et le linge de lit sont fournis.Vous séjournerez à 2,7 km de la collégiale Saint-Martin et à 4,2 km du parc des expositions Colmar Expo. L'aéroport de Bâle, le plus proche, est implanté à 59 km. "

In [24]:
# Clean location column
colmar['location'] = ["Colmar" if value!="Colmar" else value for value in colmar["location"]]

In [25]:
colmar.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Colmar,https://www.booking.com/hotel/fr/logement-peti...,"Logement Petite Venise ""appartement & parking""","9,1",Hébergement géré par un particulier,"48.07366420,7.35776200",48.07366420,7.35776200,Vous pouvez bénéficier d'une réduction Genius ...
1,Colmar,https://www.booking.com/hotel/fr/le-victor-hug...,Le Victor Hugo,"8,6",Hébergement géré par un particulier,"48.07611700,7.36353500",48.07611700,7.36353500,Vous pouvez bénéficier d'une réduction Genius ...
2,Colmar,https://www.booking.com/hotel/fr/bredala.fr.ht...,Appartement Bredala,"8,6","L'hébergement Bredala se situe à Colmar, à 100...","48.07805253,7.35612649",48.07805253,7.35612649,Vous pouvez bénéficier d'une réduction Genius ...
3,Colmar,https://www.booking.com/hotel/fr/au-brochet.fr...,AU brochet,"8,4",Hébergement géré par un particulier,"48.07406922,7.35810319",48.07406922,7.35810319,Vous pouvez bénéficier d'une réduction Genius ...
4,Colmar,https://www.booking.com/hotel/fr/cote-jardin-c...,Gite Côté Jardin Colmar,"8,8",Hébergement géré par un particulier,"48.07990300,7.36100800",48.07990300,7.36100800,Vous pouvez bénéficier d'une réduction Genius ...


In [26]:
colmar.to_csv("Colmar.csv")